In [16]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np 
import seaborn as sns
from sklearn.model_selection import train_test_split 

from sklearn.linear_model import LinearRegression 
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn import metrics 

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn import svm

from statistics import mean

import itertools
# Xây dựng mô hình đánh giá chất lượng rượu sử dụng phương pháp hồi quy tuyến tính (7 điểm)

In [17]:
def A_model(x, y):
    # Split datasets into train and test:
    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 0.7, random_state = 21)

    # Creating the linear regression model and fitting the data to it
    A_regressor = LinearRegression()
    A_regressor.fit(x_train, y_train)
    y_pred = A_regressor.predict(x_test)
    A_weight = A_regressor.coef_
    # độ lỗi huấn luyện
    A_square_error = mean_squared_error(y_test, y_pred)


    return A_weight, A_square_error

In [18]:
def B_model(x, y):
    cross_validation = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 21)
    regressor = LinearRegression()

    square_error = []
    min_square_error = float("inf")
    min_index = -1
    min_weight = 0
    for i in range(x.shape[1]): # mỗi thuộc tính của x
        x_data = x[:, i]
        for train_index, test_index in cross_validation.split(x, y): # 5 phần data sau khi chia = 4 train + 1 test
            x_train = []
            y_train = []
            for j in train_index:
                x_train.append([x_data[j]])
                y_train.append([y[j]])       
            x_train = np.array(x_train)
            y_train = np.array(y_train)

            x_test = []
            y_test = []
            for j in test_index:
                x_test.append([x_data[j]])
                y_test.append([y[j]])
            x_test = np.array(x_test)

            regressor.fit(x_train, y_train)

            y_pred = regressor.predict(x_test)
            # độ lỗi huấn luyện
            square_error.append(mean_squared_error(y_test, y_pred))

        if min_square_error > mean(square_error):
            min_square_error = mean(square_error)
            min_index = i
            min_weight = regressor.coef_
        square_error = []

    return min_square_error, min_index, min_weight

In [19]:
def C_model(x, y):
    cross_validation = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 21)
    regressor = LinearRegression()

    square_error = list()
    min_square_error = [float("inf")]
    min_index = []
    min_weight = 0
    for i in range(2, x.shape[1] - 1):
        for combination in itertools.combinations(range(2, x.shape[1] - 1), i): # tổ hợp từ 2 đến n - 1 thuộc tính của x
            x_data = []
            for c in combination:
                x_data.append(x[:, c])
            x_data = np.vstack(x_data).T
            
            for train_index, test_index in cross_validation.split(x, y): # 5 phần data sau khi chia = 4 train + 1 test
                x_train = []
                y_train = []
                for j in train_index:
                    x_train.append(x_data[j])
                    y_train.append([y[j]])       
                x_train = np.array(x_train)
                y_train = np.array(y_train)

                x_test = []
                y_test = []
                for j in test_index:
                    x_test.append(x_data[j])
                    y_test.append([y[j]])
                x_test = np.array(x_test)
                regressor.fit(x_train, y_train)

                y_pred = regressor.predict(x_test)
                # độ lỗi huấn luyện
                square_error.append(mean_squared_error(y_test, y_pred))
            if mean(min_square_error) > mean(square_error):
                min_square_error = square_error
                min_index = combination
                min_weight = regressor.coef_
            square_error = list()
    return mean(min_square_error), min_index, min_weight[0]
   

In [20]:
if __name__ == '__main__':
    df = pd.read_csv('wine.csv', sep=';') 
    x = df.iloc[:, :-1].to_numpy() # 11 cột đầu làm X
    y = df.iloc[:, -1].to_numpy()  # cột cuối cùng làm y
    A_weight, A_square_error = A_model(x, y)
  
    # displaying coefficients of each feature
    coeffecients = pd.DataFrame(A_weight, df.columns[:-1]) 
    coeffecients.columns = ['Coeffecient']
    print("Mô hình sử dụng toàn bộ 11 đặc trưng đề bài cung cấp ")
    print(coeffecients)
    print("Độ lỗi huấn luyện: ", A_square_error)

    B_square_error, B_index, B_weight = B_model(x, y)
    coeffecients = pd.DataFrame(B_weight, df.columns[B_index:B_index + 1]) 
    coeffecients.columns = ['Coeffecient']
    print("\nb) Mô hình sử dụng duy nhất 1 đặc trưng cho kết quả tốt nhất")
    print(coeffecients)
    print("Độ lỗi huấn luyện: ", B_square_error)

    C_square_error, C_index, C_weight = C_model(x, y)
    label = [df.columns[i] for i in C_index]
    # label = np.reshape(label, (len(C_index), 1))

    coeffecients = pd.DataFrame([C_weight], columns = label) 
    coeffecients = coeffecients.T
    coeffecients.columns = ['Coeffecient']
    print("\nc) Mô hình của riêng bạn cho kết quả tốt nhất")
    print(coeffecients)
    print("Độ lỗi huấn luyện: ", C_square_error)



Mô hình sử dụng toàn bộ 11 đặc trưng đề bài cung cấp 
                      Coeffecient
fixed acidity            0.044999
volatile acidity        -1.006174
citric acid             -0.200043
residual sugar           0.032139
chlorides               -1.726505
free sulfur dioxide     -0.001263
total sulfur dioxide    -0.003014
density                -40.051305
pH                      -0.182331
sulphates                0.793019
alcohol                  0.271916
Độ lỗi huấn luyện:  0.3969940582870727

b) Mô hình sử dụng duy nhất 1 đặc trưng cho kết quả tốt nhất
         Coeffecient
alcohol     0.376382
Độ lỗi huấn luyện:  0.48751785723897884

c) Mô hình của riêng bạn cho kết quả tốt nhất
                      Coeffecient
citric acid              1.160395
residual sugar           0.131154
chlorides               -3.623403
free sulfur dioxide      0.010147
total sulfur dioxide    -0.008100
density               -144.381100
sulphates                1.107484
Độ lỗi huấn luyện:  0.47778833952828